In [ ]:
from math import sqrt


# Funções básicas

## Calculo do erro

In [ ]:
def erro_absoluto(referencia, estimativa):
    return abs(referencia - estimativa)


def erro_relativo(referencia, estimativa):
    return abs((referencia, estimativa) / estimativa)


# Derivação numerica

In [ ]:
DELTA_DERIVACAO_PADRAO = 10**-4


def derivada_direita(f, delta=DELTA_DERIVACAO_PADRAO):
    return lambda x: (f(x + delta) - f(x)) / delta


def derivada_esquerda(f, delta=DELTA_DERIVACAO_PADRAO):
    return lambda x: (f(x - delta) - f(x)) / delta


def derivada_central(f, delta=DELTA_DERIVACAO_PADRAO):
    return lambda x: (f(x + delta) - f(x - delta)) / (2 * delta)


# Métodos iterativos

## Condições de parada

In [ ]:
def limite_iteracao(i):
    return lambda est: est.i >= i


def criterio_erro_absoluto(referencia, e):
    return lambda est: erro_absoluto(referencia, est.x) < e


def criterio_ab(e):
    return lambda est: est.b - est.a < 2 * e


def criterio_x(e):
    return lambda est: est.f(est.x - e) * est.f(est.x + e) < 0


def criterio_delta_x(delta):
    return lambda est: est.prior_x is not None and abs(est.x - est.prior_x) < delta


def criterio_y(e):
    return lambda est: abs(est.y) < e


## Método da bisseçao

In [ ]:
class EstatisticaAB:
    def __init__(self, f, a, b, x, y):
        self.f = f
        self.i = 1
        self.a = a
        self.b = b
        self.prior_x = None
        self.x = x
        self.y = y

    def atualizar(self, a, b, x, y):
        self.i += 1
        self.a = a
        self.b = b
        self.prior_x = x
        self.x = x
        self.y = y

    def __str__(self) -> str:
        return f"iteracao: {self.i}, a: {self.a}, b: {self.b}, x: {self.x}, y: {self.y}"


In [ ]:
def bissecao(f, a, b, criterio, observador=lambda _: None):
    if a > b:
        a, b = b, a
    i = 1
    x = (a + b) / 2
    y = f(x)
    est = EstatisticaAB(f, a, b, x, y)
    observador(est)
    while not criterio(est):
        i += 1
        if f(a) * y < 0:
            b = x
        else:
            a = x
        x = (a + b) / 2
        y = f(x)
        est_erro = (b - a) / 2
        est.atualizar(a, b, x, y)
        observador(est)
    return est


## Método da falsa posição

In [ ]:
def falsa_posicao(f, a, b, criterio, observador=lambda _: None):
    if a > b:
        a, b = b, a
    i = 1
    x = (a * f(b) - b * f(a)) / (f(b) - f(a))
    y = f(x)
    est = EstatisticaAB(f, a, b, x, y)
    observador(est)
    while not criterio(est):
        i += 1
        if f(a) * y < 0:
            b = x
        else:
            a = x
        x = (a * f(b) - b * f(a)) / (f(b) - f(a))
        y = f(x)
        est_erro = (b - a) / 2
        est.atualizar(a, b, x, y)
        observador(est)
    return est


## Método do ponto fixo

In [ ]:
class EstatisticaX:
    def __init__(self, f, fi, x, y) -> None:
        self.i = 1
        self.f = f
        self.fi = fi
        self.prior_x = None
        self.x = x
        self.y = y

    def atualizar(self, x, y):
        self.i += 1
        self.prior_x = x
        self.x = x
        self.y = y

    def __str__(self) -> str:
        return f"iteracao: {self.i}, x: {self.x}, y:{self.y}"


In [ ]:
def ponto_fixo(f, fi, x, criterio, observador=lambda _: None):
    i = 1
    y = f(x)
    est = EstatisticaX(f, fi, x, y)
    observador(est)
    while not criterio(est):
        i += 1
        x = fi(x)
        y = f(x)
        est.atualizar(x, y)
        observador(est)
    return est


## Método de Newtown-Raphson

In [ ]:
def newton_raphson(f, x, criterio, observador=lambda x: None):
    d = derivada_central(f)
    fi = lambda x: x - f(x) / d(x)
    return ponto_fixo(f, fi, x, criterio, observador)


# Amostras

In [ ]:
def amostra_linear(x):
    return 5 * x + 2


def amostra_poli(x):
    return x**2 + x - 6


def amostra_poli_iter(x):
    return sqrt(6 - x)


# Playground 

In [ ]:
resultado = newton_raphson(amostra_poli, 1.5, criterio_x(10**-4))
print(resultado)
